In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import Xception
import pandas as pd
import os
import json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

Number of devices: 1


In [3]:
tf.random.set_seed(42)

In [4]:
BATCH_SIZE = 32
EPOCHS = 50
INITIAL_LEARNING_RATE = 0.0001
IMAGE_SIZE = (299, 299)

In [5]:
data_dir = '/kaggle/input/finalskindiseasedataset/dataset'

image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    for image_file in os.listdir(label_dir):
        image_paths.append(os.path.join(label_dir, image_file))
        labels.append(label)

X_train, X_temp, y_train, y_temp = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.xception.preprocess_input
)
val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input
)

In [7]:
train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_train, 'label': y_train}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_val, 'label': y_val}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 19556 validated image filenames belonging to 24 classes.
Found 2445 validated image filenames belonging to 24 classes.


In [8]:
base_model = Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(299, 299, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024,activation = 'relu')(x)
output = Dense(24, activation='softmax')(x)



model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=INITIAL_LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


83683744/83683744 [==============================] - 3s 0us/step


In [9]:
"""
x = GlobalAveragePooling2D()(base_model.layers[-1].output)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(24, activation='softmax')(x)  # 24 classes

# Build the custom classification head
x = GlobalAveragePooling2D()(base_model.layers[-1].output)
x = BatchNormalization()(x)
x = Dense(1024)(x)
x = LeakyReLU()(x)  # LeakyReLU activation
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(24, activation='softmax')(x)  # 24 classes
"""

"\nx = GlobalAveragePooling2D()(base_model.layers[-1].output)\nx = BatchNormalization()(x)\nx = Dense(512, activation='relu')(x)\nx = Dropout(0.5)(x)\nx = BatchNormalization()(x)\noutput = Dense(24, activation='softmax')(x)  # 24 classes\n\n# Build the custom classification head\nx = GlobalAveragePooling2D()(base_model.layers[-1].output)\nx = BatchNormalization()(x)\nx = Dense(1024)(x)\nx = LeakyReLU()(x)  # LeakyReLU activation\nx = BatchNormalization()(x)\nx = Dropout(0.5)(x)\nx = Dense(512)(x)\nx = LeakyReLU()(x)\nx = BatchNormalization()(x)\nx = Dropout(0.5)(x)\noutput = Dense(24, activation='softmax')(x)  # 24 classes\n"

In [10]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [11]:
def lr_schedule(epoch):
    if epoch < 5:
        return INITIAL_LEARNING_RATE
    else:
        return INITIAL_LEARNING_RATE * tf.math.exp(0.1 * (10 - epoch))

model_checkpoint = ModelCheckpoint(
    "best_model.h5",
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(X_val) // BATCH_SIZE,
    callbacks=[
        LearningRateScheduler(lr_schedule),
        early_stopping,
        model_checkpoint
    ]
)

Epoch 1/50
114/611 [====>.........................] - ETA: 8:21 - loss: 3.0480 - accuracy: 0.1335

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input
)

test_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_test, 'label': y_test}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

best_model = tf.keras.models.load_model("best_model.h5")

In [ ]:
best_model.save("xception_custom_classifier.h5")

model_json = model.to_json()
with open("xception_custom_classifier.json", "w") as json_file:
    json_file.write(model_json)